## 4 Smoothing and Interpolation
You have been given two corpora: “Pride and Prejudice” and “Ulysses”. Your
task is to design Language Models for both using smoothing techniques. Be
sure to use the tokenizer created in the Tokenization task.
1. Create language models with the following parameters:
- (a).  On “Pride and Prejudice” corpus:
- i. LM 1: Tokenization + 3-gram LM + Good-Turing Smoothing
- ii. LM 2: Tokenization + 3-gram LM + Linear Interpolation
- (b) On “Ulysses” corpus:
- i. LM 3: Tokenization + 3-gram LM + Good-Turing Smoothing
- ii. LM 4: Tokenization + 3-gram LM + Linear Interpolation
2. For each of these corpora, create a test set by randomly selecting 1000
sentences. This set will not be used for training the LM.
- (a) Calculate perplexity score for each sentence of “Pride and Prejudice”
corpus and “Ulysses” corpus for each of the above models
and also get average perplexity score on the train corpus.
- (b) Report the perplexity scores for all the sentences in the training
set. Report the perplexity score on the test sentences as well, in
the same manner above
- Note: Good-Turing Smoothing is a technique used to adjust the probability
distribution of unseen events in N-gram models, 
- while Linear - Interpolation
is a method of combining different N-gram models (like unigram,
bigram, trigram) to get a better estimate of the probabilities. For more
details, refer to the resources section at the end.

# Focus on good turing

In [123]:
import re
import random
def test_train_split(corpus, n):
    # remove new line
    corpus = corpus.replace('\n', ' ')
    # split into sentences
    sentences = re.split(r'(?<=[.!?]) +', corpus)
    test_sentences = random.sample(sentences, n)
    train_sentences = [sentence for sentence in sentences if sentence not in test_sentences]
    return test_sentences, train_sentences


def tokenize(text):
    url_pattern1 = "(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"
    url_pattern2 = r'www\.[^\s\.]+(?:\.[^\s\.]+)*(?:[\s\.]|$)'
    email_pattern = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    mention_pattern = "@\w+"
    hastag_pattern = "#[a-z0-9_]+"
    normal_pattern = "[a-zA-Z]+"
    number_pattern = "[0-9]+"
    pattern = r'[,"\'()\[\]{}-]'  # Matches , " ' ( ) [ ] { }
    tokens = []
    text = text.lower()
    text = re.sub(pattern, '', text)
    text = re.sub(url_pattern1, '<URL> ', text)
    text = re.sub(url_pattern2, '<URL> ', text)
    text = re.sub(email_pattern, '<MAILID> ', text)
    text = re.sub(hastag_pattern, '<HASHTAG> ', text)
    text = re.sub(mention_pattern, '<MENTION> ', text)
    text = re.sub(number_pattern, '<NUM> ', text)
    tokens = re.findall(r'\b\w+|[^\s\w<>]+|<\w+>', text)
    return tokens


coupus_path = './corpus'
corpus1 = "./corpus/Pride and Prejudice - Jane Austen.txt"
corpus2 = "./corpus/Ulysses  James Joyce.txt"
with open(corpus1, 'r') as f:
    text1 = f.read()
test_sentences, train_sentences = test_train_split(text1, 1000)

In [124]:
def perform_cleaning(text):
    # remove comma, extra spaces, and punctuations
    text = re.sub(r'[,!?;-]+', '', text)
    if text.endswith('.'):
            text = text[:-1]#removing last dot also
    return text
def PerformNgram(corpus, n):
    pattern = "(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
    list_sentences = re.split(pattern, corpus)
    ngrams = {}
    for sentence in list_sentences:
        tokens = tokenize(sentence)
        # sentence = (n-1)*"<START> "+ sentence
        for i in range(len(tokens)-n+1):
            temp = tuple(tokens[j] for j in range(i, i+n))  # Convert list to tuple
            if temp in ngrams:
                ngrams[temp] += 1
            else:
                ngrams[temp] = 1
            
    return ngrams

In [145]:
def freq_of_freq(train_trigram_count):
    freq_of_freq_trigram = {}
    for _, count in train_trigram_count.items():
        freq_of_freq_trigram[count] = freq_of_freq_trigram.get(count, 0) + 1
    return freq_of_freq_trigram


unigram_count = PerformNgram(" ".join(train_sentences), 1)
bigram_count = PerformNgram(" ".join(train_sentences), 2)
train_trigram_count = PerformNgram(" ".join(train_sentences), 3)
freq_of_freq_trigram = freq_of_freq(train_trigram_count)
train_trigram_count

{('the', 'project', 'gutenberg'): 16,
 ('project', 'gutenberg', 'ebook'): 3,
 ('gutenberg', 'ebook', 'pride'): 3,
 ('ebook', 'pride', 'and'): 3,
 ('pride', 'and', 'prejudice'): 6,
 ('and', 'prejudice', 'by'): 1,
 ('prejudice', 'by', 'jane'): 1,
 ('by', 'jane', 'austen'): 1,
 ('jane', 'austen', 'edited'): 1,
 ('austen', 'edited', 'by'): 1,
 ('edited', 'by', 'r'): 1,
 ('by', 'r', '.'): 1,
 ('robert', 'william', 'chapman'): 2,
 ('william', 'chapman', 'this'): 1,
 ('chapman', 'this', 'ebook'): 1,
 ('this', 'ebook', 'is'): 1,
 ('ebook', 'is', 'for'): 1,
 ('is', 'for', 'the'): 1,
 ('for', 'the', 'use'): 2,
 ('the', 'use', 'of'): 3,
 ('use', 'of', 'anyone'): 1,
 ('of', 'anyone', 'anywhere'): 1,
 ('anyone', 'anywhere', 'at'): 1,
 ('anywhere', 'at', 'no'): 1,
 ('at', 'no', 'cost'): 1,
 ('no', 'cost', 'and'): 1,
 ('cost', 'and', 'with'): 1,
 ('and', 'with', 'almost'): 1,
 ('with', 'almost', 'no'): 1,
 ('almost', 'no', 'restrictions'): 1,
 ('no', 'restrictions', 'whatsoever'): 1,
 ('restrictions'

In [208]:
import numpy as np
from sklearn.linear_model import LinearRegression
def findLinearRegression(freq_of_freq_trigram):
    Nc_values = np.array(list(freq_of_freq_trigram.keys())).reshape(-1, 1)  # Features (Nc)
    c_values = np.array(list(freq_of_freq_trigram.values())).reshape(-1, 1)  # Target variable (c)
    # Fit linear regression model
    # Log-transform Nc and c values
    log_Nc_values = np.log(Nc_values)
    log_c_values = np.log(c_values)
    # Fit linear regression model to log-transformed data
    model = LinearRegression()
    model.fit(log_c_values, log_Nc_values)
    return model
model = findLinearRegression(freq_of_freq_trigram)
#predict model first
#Finding modified count for smoothing good turing
modified_count = {}
for count, Nc in freq_of_freq_trigram.items():
    Nc_1 = freq_of_freq_trigram.get(count+1, 0)
    if Nc_1 == 0:
        # use model now
        log_c_new = np.log([[count + 1]])
        log_Nc_predicted = model.predict(log_c_new)
        Nc_predicted = np.exp(log_Nc_predicted)
        c_new = (count + 1) * Nc_predicted / Nc
        print(count, Nc, c_new, Nc_predicted)
        break
    else:
        c_new = (count + 1) * Nc_1 / Nc
    modified_count[count] = c_new



51 2 [[260.95184152]] [[10.03660929]]


In [154]:
# Pc = ((count(w1w2w3) + 1) * N(count(w1w2w3) + 1)) / (count(w1w2w3))
# if N(count(w1w2w3) + 1) is 0 then 
#  for all we have to find Zc = N(count(w1w2w3)) / (0.5 * (t - q)) so we can use it later for those whoe 
# val becomes 0
# where t = count(w1w2w3) + 1 and q = count(w1w2w3) - 1
# if(count(w1w2w3) + 1 == 0) then t - q = count(w1w2w3) - q
# if(count(w1w2w3) + 1 == 0) then t - q = t - count(w1w2w3)
# After getting Zc, using linear regressiong to get Pc
# predicting Pc with linear log Zc vs log Count of all triagrams

# for train finding perplexity
Zc_train = {}
for trigram, count in train_trigram_count.items():
    # smoothing

    c_plus_1 = count + 1
    c_minus_1 = count - 1
    N_c_plus_1 = freq_of_freq_trigram.get(c_plus_1, 0)
    N_c = freq_of_freq_trigram.get(count, 0)
    if N_c_plus_1 == 0:
        p = c_plus_1
        q = c_minus_1
        if train_trigram_count.get(p, 0) != 0 and train_trigram_count.get(q, 0) != 0:
            Zc = N_c / (0.5 * (p - q))
        elif train_trigram_count.get(q, 0) != 0 :
            Zc = N_c / (0.5 * (p - count))
        elif train_trigram_count.get(p, 0) == 0:
            Zc = N_c / (0.5 * (count - q))
        else:
            Zc = N_c / (0.5 * (count))
        Zc_train[trigram] = Zc
# go for testing part



{('mr', '.', 'bennet'): 4.0, ('that', 'he', 'had'): 2.0, ('as', 'soon', 'as'): 2.0, ('mr', '.', 'bingley'): 2.0, ('i', 'do', 'not'): 2.0, ('i', 'am', 'sure'): 4.0, (';', 'and', 'after'): 16.0, ('of', 'mr', '.'): 2.0, ('as', 'much', 'as'): 16.0, ('the', 'rest', 'of'): 16.0, ('she', 'could', 'not'): 2.0, ('mr', '.', 'darcy'): 2.0, ('and', 'mr', '.'): 2.0, ('could', 'not', 'help'): 16.0, ('the', 'room', '.'): 16.0, ('not', 'to', 'be'): 16.0, (';', 'but', 'she'): 16.0, ('mr', '.', 'collins'): 2.0, ('mr', '.', 'wickham'): 2.0, ('*', '*', '*'): 16.0}


In [165]:
def good_turing_smoothing(trigram, train_trigram_count,freq_of_freq_trigram):
    if train_trigram_count.get(trigram, 0) == 0:
        c_plus_1 = 1
        c_minus_1 = 0
        N_c_plus_1 = freq_of_freq_trigram.get(c_plus_1, 0)
        N_c = len(train_trigram_count)
        return N_c_plus_1/N_c
    else:

    return 555


In [166]:
import math
def perplexity_good_turing(sentence, train_trigram_count, freq_of_freq_trigram):
    tokens = tokenize(sentence)
        # in this tuple add <start> <start> at the start of the sentence and <end> at the end of the sentence
    tokens = ('<START>', '<START>',) + tuple(tokens) + ('<END>',)
    # break  
    log_probability_sum = 0.0
    trigram_count = 0
    for i in range(len(tokens)-3):
        trigram = tuple(tokens[i:i+3])
        trigram_count += 1
        temp_prob = math.log(good_turing_smoothing(trigram, train_trigram_count,freq_of_freq_trigram))
        # log_probability_sum += math.log(linear_interpolation(trigram, unigram_prob, bigram_prob, trigram_prob))
        log_probability_sum += temp_prob

    sentence_perplexity = math.exp(-(log_probability_sum / trigram_count))
    return sentence_perplexity

# Now I have to find the perplexity
test_perplexity = {}#perplexity for each sentence
for sentence in test_sentences:
    test_perplexity[sentence] = perplexity_good_turing(sentence, train_trigram_count,freq_of_freq_trigram)


In [127]:
# def calculate_trigram_probability(sentence, trigram_counts, bigram_counts):
#     # Tokenize the sentence into words
#     # Pad the sentence with start and end tokens
#     sentence = (3-1)*"<START> "+ sentence
#     # Initialize the probability
#     probability = 1.0
#     padded_sentence = sentence.split(' ')
#     # Calculate the probability of each word given its preceding bigram
#     for i in range(4, len(padded_sentence)):
#         trigram = (padded_sentence[i-2:i+1])
#         bigram = trigram[:2]
#         trigram = ' '.join(trigram)
#         bigram = ' '.join(bigram)
#         word = padded_sentence[i]
#         # Get counts from the training data
#         trigram_count = trigram_counts.get(trigram, 0)
#         bigram_count = bigram_counts.get(bigram, 0)
#         if bigram_count == 0:
#             return 0
#         probability_word = trigram_count / bigram_count
#         probability *= probability_word   
    
#     return probability

# # Example usage:
# sentence_probability = {}
# for sentence in train_sentences:
#     # sentence = "finding the probability of each sentence"
#     sentence_probability[sentence] = calculate_trigram_probability(sentence, trigram_count, bigram_count)
#     break


AttributeError: 'list' object has no attribute 'get'

# Now finding smoothing for each sentence
If that is present then ditectly find the value otherwise have to find Zc

In [ ]:
# Pc = ((count(w1w2w3) + 1) * N(count(w1w2w3) + 1)) / (count(w1w2w3))
# if N(count(w1w2w3) + 1) is 0 then 
#  for all we have to find Zc = N(count(w1w2w3)) / (0.5 * (t - q)) so we can use it later for those whoe 
# val becomes 0
# where t = count(w1w2w3) + 1 and q = count(w1w2w3) - 1
# if(count(w1w2w3) + 1 == 0) then t - q = count(w1w2w3) - q
# if(count(w1w2w3) + 1 == 0) then t - q = t - count(w1w2w3)
# After getting Zc, using linear regressiong to get Pc
# predicting Pc with linear log Zc vs log Count of all triagrams

trigram_count
    break

['the', 'project', 'gutenberg', 'ebook', 'pride', 'and', 'prejudice', 'by', 'jane', 'austen', 'edited', 'by', 'r', '.']


In [ ]:
class N_Gram_Model:
    def __init__(self) -> None:
        pass
    def read_file(file_path):
        with open(file_path, 'r') as f:
            text = f.read()
        return text
    def test_train_split(corpus, n):
        # remove new line
        corpus = corpus.replace('\n', ' ')
        # split into sentences
        sentences = re.split(r'(?<=[.!?]) +', corpus)
        test_sentences = random.sample(sentences, n)
        train_sentences = [sentence for sentence in sentences if sentence not in test_sentences]
        return test_sentences, train_sentences
    
    def train():
        pass
    def test():
        pass
    def save():
        pass
    def load(): 
        pass
    def perplexity():
        # it is required to calculate perplexity of train and test data
        pass
    def generate():
        # genearate a sentence
        # use probability calculated in train to generate next word
        pass

    def evaluate():
        # evaluate the model
        # calculate perplexity of train and test data
        pass

    def good_turing():
        # implement good turing smoothing
        pass

    def interpolation():
        # implement interpolation smoothing
        pass


In [ ]:
# import nltk
# from nltk.tokenize import RegexpTokenizer
# from nltk.tokenize import word_tokenize
import re
import random
import sys
import math
# nltk.download('punkt')

#maps 
ngram_context_counter = {}
ngram_continue_counter = {}
words_preceding_ngram = {}
words_prec_and_infront_ngram = {}
unique_words_preceding_ngram = {}
ngram_continue_counter_dict = {}

#helper funcitons 
def test_train_split(text ,n):
    text = re.sub(r"\n", ' ', text)
    sentences = re.split(r'(?<=[.!?]) +', text)
    test_sentences = random.sample(sentences, n)
    train_sentences = [sentence for sentence in sentences if sentence not in test_sentences]
    return test_sentences, train_sentences

def tokenize(text):
    url_re = "(http|ftp|https):\/\/([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:\/~+#-]*[\w@?^=%&\/~+#-])"
    mention_re = "@\w+"
    hastag_re = "#[a-z0-9_]+"
    text = text.lower()
    text = re.sub(url_re, '<url> ', text)
    text = re.sub(hastag_re, '<hashtag> ', text)
    text = re.sub(mention_re, '<mention> ', text)
    tokens = re.findall(r'\b\w+|[^\s\w<>]+|<\w+>', text)
    return tokens

def generate_ngrams(tokens, n):
    tokens = (n-1)*['<START>']+tokens
    print(tokens)
    ngrams = [(tuple(tokens[i-p-1] for p in reversed(range(n-1))), tokens[i]) for i in range(n-1, len(tokens))]
    return ngrams

def gen_context_counter(ngrams):
    ngram_context = {}
    ngram_counter = {}

    for ngram in ngrams:
        if ngram in ngram_counter:
            ngram_counter[ngram] += 1
        else:
            ngram_counter[ngram] = 1
        
        prev_words, target_word = ngram
        
        if prev_words in ngram_context:
            ngram_context[prev_words].append(target_word)
        else:
            ngram_context[prev_words] = [target_word]
    
    return ngram_context, ngram_counter
